# YOLOv7 Door Plaque Detection



Environment Setup

In [ ]:
# Verify GPU
!nvidia-smi

In [ ]:
# Navigate to content directory and set up workspace
%cd /content

# Remove any existing yolov7 folder
!rm -rf yolov7

print("Workspace prepared")

Upload Dataset



In [ ]:
from google.colab import files
import zipfile
import os

# Upload the zip file
print("Please upload your dataset ZIP file...")
uploaded = files.upload()

if uploaded:
    zip_filename = list(uploaded.keys())[0]
    print(f"\n Extracting {zip_filename}...")

    # Extract
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('/content/dataset')

    # Find actual dataset path
    dataset_contents = os.listdir('/content/dataset')
    if len(dataset_contents) == 1 and os.path.isdir(f'/content/dataset/{dataset_contents[0]}'):
        DATASET_PATH = f'/content/dataset/{dataset_contents[0]}'
    else:
        DATASET_PATH = '/content/dataset'

    print(f"Dataset extracted to: {DATASET_PATH}")
else:
    print("No file uploaded. Please run this cell again.")

Dataset


In [ ]:
import os

# Check dataset exists
if not os.path.exists(DATASET_PATH):
    print(f"ERROR: Dataset not found at {DATASET_PATH}")
else:
    print(f"Dataset found at: {DATASET_PATH}\n")

    # Count images
    def count_images(folder_path):
        if not os.path.exists(folder_path):
            return 0
        return len([f for f in os.listdir(folder_path)
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

    train_imgs = count_images(os.path.join(DATASET_PATH, 'train/images'))
    val_imgs = count_images(os.path.join(DATASET_PATH, 'valid/images'))
    test_imgs = count_images(os.path.join(DATASET_PATH, 'test/images'))

    print("Dataset Summary:")
    print(f"  Training:   {train_imgs} images")
    print(f"  Validation: {val_imgs} images")
    print(f"  Test:       {test_imgs} images")
    print(f"  Total:      {train_imgs + val_imgs + test_imgs} images")

    if train_imgs == 472 and val_imgs == 100 and test_imgs == 20:
        print("\n Image counts match expected values!")

In [ ]:
# Clone YOLOv7
%cd /content
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7

print("YOLOv7 cloned")

In [ ]:
# Install dependencies
!pip install -q matplotlib>=3.2.2
!pip install -q numpy>=1.18.5
!pip install -q opencv-python>=4.1.2
!pip install -q Pillow>=7.1.2
!pip install -q PyYAML>=5.3.1
!pip install -q requests>=2.23.0
!pip install -q scipy>=1.4.1
!pip install -q tqdm>=4.41.0
!pip install -q tensorboard>=2.4.1
!pip install -q pandas>=1.1.4
!pip install -q seaborn>=0.11.0

print("Dependencies installed")

In [ ]:
import os
import re

def fix_torch_load(filepath):
    """Add weights_only=False to torch.load calls"""
    if not os.path.exists(filepath):
        return False

    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

    original = content

    patterns = [
        (r"torch\.load\(([^,]+), map_location=([^)]+)\)(?!.*weights_only)",
         r"torch.load(\1, map_location=\2, weights_only=False)"),
        (r"torch\.load\(cache_path\)(?!.*weights_only)",
         r"torch.load(cache_path, weights_only=False)"),
        (r"torch\.load\(f, map_location=torch\.device\('cpu'\)\)(?!.*weights_only)",
         r"torch.load(f, map_location=torch.device('cpu'), weights_only=False)"),
    ]

    for pattern, replacement in patterns:
        content = re.sub(pattern, replacement, content)

    if content != original:
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)
        return True
    return False

files_to_fix = [
    'train.py',
    'test.py',
    'detect.py',
    'models/experimental.py',
    'utils/datasets.py',
    'utils/general.py'
]


for filepath in files_to_fix:
    if fix_torch_load(filepath):
        print(f"Fixed: {filepath}")
    else:
        print(f"⏭Skipped: {filepath}")

In [ ]:
# Download pre-trained weights
!wget -q https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

if os.path.exists('yolov7.pt'):
    print("Downloaded yolov7.pt")
else:
    print("Failed to download weights")

In [ ]:
import yaml

data_yaml_path = os.path.join(DATASET_PATH, 'data.yaml')

if os.path.exists(data_yaml_path):
    # Read current config
    with open(data_yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)

    print("Current data.yaml:")
    print(yaml.dump(data_config, default_flow_style=False))

    # Update to absolute paths
    data_config['train'] = os.path.join(DATASET_PATH, 'train/images')
    data_config['val'] = os.path.join(DATASET_PATH, 'valid/images')
    data_config['test'] = os.path.join(DATASET_PATH, 'test/images')

    # Save updated config
    with open(data_yaml_path, 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False)

    print("\n Updated data.yaml with absolute paths")
    print("\nNew configuration:")
    print(yaml.dump(data_config, default_flow_style=False))
else:
    print(f"data.yaml not found at {data_yaml_path}")

In [ ]:
# Training configuration
EPOCHS = 75           # Number of epochs
BATCH_SIZE = 16        # Batch size
IMG_SIZE = 640         # Image size
WEIGHTS = 'yolov7.pt'  # Pre-trained weights
PROJECT_NAME = 'door_plaque_detector'

print("Training Configuration:")
print(f"  Epochs:      {EPOCHS}")
print(f"  Batch Size:  {BATCH_SIZE}")
print(f"  Image Size:  {IMG_SIZE}x{IMG_SIZE}")
print(f"  Pre-trained: {WEIGHTS}")
print(f"  Project:     {PROJECT_NAME}")

# Disable wandb
import os
os.environ['WANDB_DISABLED'] = 'true'
print("\n Configuration ready")

Train Model

In [ ]:
# Start training
!python train.py \
    --workers 8 \
    --device 0 \
    --batch-size {BATCH_SIZE} \
    --epochs {EPOCHS} \
    --img {IMG_SIZE} {IMG_SIZE} \
    --data {data_yaml_path} \
    --hyp data/hyp.scratch.p5.yaml \
    --cfg cfg/training/yolov7.yaml \
    --weights {WEIGHTS} \
    --name {PROJECT_NAME} \
    --cache-images

In [ ]:
import glob

# Find the trained model
train_dirs = glob.glob(f'runs/train/{PROJECT_NAME}*')

if train_dirs:
    # Get the most recent
    latest_dir = max(train_dirs, key=os.path.getctime)
    BEST_WEIGHTS = os.path.join(latest_dir, 'weights/best.pt')
    LAST_WEIGHTS = os.path.join(latest_dir, 'weights/last.pt')

    # Convert to absolute path
    BEST_WEIGHTS = os.path.abspath(BEST_WEIGHTS)
    LAST_WEIGHTS = os.path.abspath(LAST_WEIGHTS)

    print(f" Training completed!\n")
    print(f" Results directory: {latest_dir}")
    print(f" Best weights: {BEST_WEIGHTS}")
    print(f" Last weights: {LAST_WEIGHTS}")

    # Verify files exist
    if os.path.exists(BEST_WEIGHTS):
        print("\n Weights file verified!")
    else:
        print("\n Warning: Weights file not found")
else:
    print(" No training runs found. Please complete training first.")

In [ ]:
from IPython.display import Image, display

if 'latest_dir' in locals():
    print(" Training Results:\n")

    results_img = os.path.join(latest_dir, 'results.png')
    if os.path.exists(results_img):
        display(Image(filename=results_img, width=900))
    else:
        print("Results plot not found")

In [ ]:
# Show confusion matrix
if 'latest_dir' in locals():
    print(" Confusion Matrix:\n")

    confusion_img = os.path.join(latest_dir, 'confusion_matrix.png')
    if os.path.exists(confusion_img):
        display(Image(filename=confusion_img, width=600))
    else:
        print("Confusion matrix not found")

In [ ]:
# Run evaluation on test set
if 'BEST_WEIGHTS' in locals() and os.path.exists(BEST_WEIGHTS):
    !python test.py \
        --data {data_yaml_path} \
        --img 640 \
        --batch 16 \
        --conf 0.001 \
        --iou 0.65 \
        --device 0 \
        --weights {BEST_WEIGHTS} \
        --name test_evaluation \
        --verbose
else:
    print("Weights not found. Please complete training first.")

Run Inference on Test Images

In [ ]:
# Detect on test images
if 'BEST_WEIGHTS' in locals() and os.path.exists(BEST_WEIGHTS):
    test_images = os.path.join(DATASET_PATH, 'test/images')

    !python detect.py \
        --weights {BEST_WEIGHTS} \
        --conf 0.25 \
        --img-size 640 \
        --source {test_images} \
        --name test_detections \
        --save-txt \
        --save-conf

    print("\n Detections saved to: runs/detect/test_detections/")
else:
    print(" Weights not found. Please complete training first.")

In [ ]:
# Display sample detections
import glob
from IPython.display import Image, display

detection_dir = 'runs/detect/test_detections'
if os.path.exists(detection_dir):
    detection_imgs = glob.glob(f'{detection_dir}/*.jpg') + glob.glob(f'{detection_dir}/*.png')

    print(f" Showing 5 sample detections (out of {len(detection_imgs)} total):\n")

    for img_path in detection_imgs[:5]:
        print(f"\n{os.path.basename(img_path)}:")
        display(Image(filename=img_path, width=700))
else:
    print("No detections found. Run the detection cell above first.")

Download Trained Model

In [ ]:
from google.colab import files

if 'BEST_WEIGHTS' in locals() and os.path.exists(BEST_WEIGHTS):
    print(" Downloading trained model...\n")
    files.download(BEST_WEIGHTS)
    print("\n Model downloaded!")
    print(f"\nYou can use this model file for inference in your applications.")
else:
    print(" No weights to download. Complete training first.")